# crowling systemを作る

In [5]:
import requests
import lxml.html

import time  # timeモジュールをインポートする。
import re

import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials

In [6]:
response = requests.get('https://gihyo.jp/dp')

In [8]:
root = lxml.html.fromstring(response.content)

In [10]:
for a in root.cssselect('a[itemprop="url"]'):
    url = a.get('href')
    print(url)

/dp/ebook/2021/978-4-297-12386-4
/dp/ebook/2021/978-4-297-12480-9
/dp/ebook/2021/978-4-297-12478-6
/dp/ebook/2021/978-4-297-12503-5
/dp/ebook/2021/978-4-297-12482-3
/dp/ebook/2021/978-4-297-12428-1
/dp/ebook/2021/978-4-297-12472-4
/dp/ebook/2021/978-4-297-12474-8
/dp/ebook/2021/978-4-297-12476-2
/dp/ebook/2021/978-4-297-12464-9
/dp/ebook/2021/978-4-297-12322-2
/dp/ebook/2021/978-4-297-12410-6
/dp/ebook/2021/978-4-297-12424-3
/dp/ebook/2021/978-4-297-12426-7
/dp/ebook/2021/978-4-297-12436-6
/dp/ebook/2021/978-4-297-12370-3
/dp/ebook/2021/978-4-297-12356-7
/dp/ebook/2021/978-4-297-12360-4
/dp/ebook/2021/978-4-297-12330-7
/dp/ebook/2021/978-4-297-12444-1
/dp/ebook/2021/978-4-297-12384-0
/dp/ebook/2021/978-4-297-12368-0
/dp/ebook/2021/978-4-297-12364-2
/dp/ebook/2021/978-4-297-11977-5
/dp/ebook/2021/978-4-297-12346-8
/dp/ebook/2021/978-4-297-12373-4
/dp/ebook/2021/978-4-297-12362-8
/dp/ebook/2021/978-4-297-12348-2
/dp/ebook/2021/978-4-297-12338-3
/dp/ebook/2021/978-4-297-12366-6
/dp
/dp/my

In [12]:
root.make_links_absolute(response.url)

In [14]:
for a in root.cssselect('#listBook a[itemprop="url"]'):
    url = a.get('href')
    print(url)

https://gihyo.jp/dp/ebook/2021/978-4-297-12386-4
https://gihyo.jp/dp/ebook/2021/978-4-297-12480-9
https://gihyo.jp/dp/ebook/2021/978-4-297-12478-6
https://gihyo.jp/dp/ebook/2021/978-4-297-12503-5
https://gihyo.jp/dp/ebook/2021/978-4-297-12482-3
https://gihyo.jp/dp/ebook/2021/978-4-297-12428-1
https://gihyo.jp/dp/ebook/2021/978-4-297-12472-4
https://gihyo.jp/dp/ebook/2021/978-4-297-12474-8
https://gihyo.jp/dp/ebook/2021/978-4-297-12476-2
https://gihyo.jp/dp/ebook/2021/978-4-297-12464-9
https://gihyo.jp/dp/ebook/2021/978-4-297-12322-2
https://gihyo.jp/dp/ebook/2021/978-4-297-12410-6
https://gihyo.jp/dp/ebook/2021/978-4-297-12424-3
https://gihyo.jp/dp/ebook/2021/978-4-297-12426-7
https://gihyo.jp/dp/ebook/2021/978-4-297-12436-6
https://gihyo.jp/dp/ebook/2021/978-4-297-12370-3
https://gihyo.jp/dp/ebook/2021/978-4-297-12356-7
https://gihyo.jp/dp/ebook/2021/978-4-297-12360-4
https://gihyo.jp/dp/ebook/2021/978-4-297-12330-7
https://gihyo.jp/dp/ebook/2021/978-4-297-12444-1
https://gihyo.jp/dp/

In [16]:
def normalize_spaces(s):
    return re.sub(r'\s+', ' ', s).strip()


In [17]:
def scrape_detail_page(response):
    """
    詳細ページのResponseから電子書籍の情報をdictで取得する。
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('#bookTitle')[0].text_content(),
        'price': root.cssselect('.buy')[0].text.strip(),
        'content': [normalize_spaces(h3.text_content()) for h3 in root.cssselect('#content > h3')],  # 目次
    }
    return ebook

In [18]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)

    for a in root.cssselect('#listBook a[itemprop="url"]'):
        url = a.get('href')
        yield url

In [26]:
def main():
    session = requests.Session()
    response = session.get('https://gihyo.jp/dp')
    urls = scrape_list_page(response)
    titles = []
    for url in urls:
        time.sleep(1)  # 1秒のウェイトを入れる
        response = session.get(url)
        ebook = scrape_detail_page(response)
        titles.append(ebook['title'])
        print(ebook)
    return titles

In [28]:
titles = main()

{'url': 'https://gihyo.jp/dp/ebook/2021/978-4-297-12386-4', 'title': '急成長を導くマネージャーの型～地位・権力が通用しない時代の“イーブン”なマネジメント', 'price': '2,280円', 'content': ['はじめに', '序章 マネジメントは経験でもセンスでもない，「型」を身につけ実行するのみ', '1章 マネージャーの役割を認識する', '2章 正確で素早い現状把握でロケットスタート', '3章 チームの役割，目標，意義を設定する', '4章 チームの戦略3点セット［方針・KPI・重要アクション］', '5章 強いチームをつくる', '6章 戦略と組織を動かす「推進システム」を作る', '7章 初期の成果とモメンタムをつくりだす', '8章 改善で継続的に成果を出し続ける', '9章 個人目標設定で成長のきっかけを与え，評価で努力に報いる', '10章 ピープルマネジメントでメンバーを動かす', '11章 3つのコミュニケーション技術を使いこなす', '12章 マネージャーの立ち位置と心得', '終章 マネージャーにとって一番大事なこと']}
{'url': 'https://gihyo.jp/dp/ebook/2021/978-4-297-12480-9', 'title': 'ゼロからはじめるiPhone 12 スマートガイド iOS 15対応版', 'price': '1,080円', 'content': ['Chapter 1 iPhoneのキホン', 'Chapter 2 電話機能を使う', 'Chapter 3 基本設定を行う', 'Chapter 4 メール機能を利用する', 'Chapter 5 インターネットを楽しむ', 'Chapter 6 音楽や写真・動画を楽しむ', 'Chapter 7 アプリを使いこなす', 'Chapter 8 iCloudを活用する', 'Chapter 9 iPhoneをもっと使いやすくする', 'Chapter 10 iPhoneを初期化・再設定する']}
{'url': 'https://gihyo.jp/dp/ebook/2021/978-4-297-12478-6', 'title': 'ゼロからはじめるiPhone 

{'url': 'https://gihyo.jp/dp/ebook/2021/978-4-297-12346-8', 'title': 'Amazon出店の王道～中小企業の新販路 その施策とアイデア', 'price': '1,680円', 'content': ['第1章 Amazon出店の基本', '第2章 出品前に決めておくこと', '第3章 Amazonに商品を出品する', '第4章 FBA納品と自社発送の方法', '第5章 売上拡大のための必修テクニック', '第6章 Amazon広告による商品露出の増やしかた', '第7章 外部マーケティング施策と作業の外注化', '第8章 トラブル対応 Q＆A']}
{'url': 'https://gihyo.jp/dp/ebook/2021/978-4-297-12373-4', 'title': '秒速決算～スピーディに人を動かす管理会計で最高の利益体質をつくる！～', 'price': '1,800円', 'content': ['第１章 毎月の数値検討を早くすると儲けが増える', '第２章 「秒速決算」を導入する', '第３章 経理以外の社員が営業利益を集計できるようになるために', '第４章 業績のベンチマーク「予算」の作り方', '第５章 経営者が「秒速決算」を活用することで手にするもの', '第６章 未上場会社のオーナー社長の皆様への言づて']}
{'url': 'https://gihyo.jp/dp/ebook/2021/978-4-297-12362-8', 'title': 'EDIUS X Pro パーフェクトガイド［改訂2版］', 'price': '3,600円', 'content': ['CHAPTER00 インストール', 'CHAPTER01 起動とプロジェクト設定', 'CHAPTER 02 クリップ管理', 'CHAPTER 03 クリップ編集', 'CHAPTER 04 トリミング', 'CHAPTER 05 トランジション', 'CHAPTER 06 エフェクト', 'CHAPTER 07 マーカー', 'CHAPTER 08 タイトル', 'CHAPTER 09 オーディオ', 'CHAPTER 10 出力', 'CHAPTER 11 HDRとLog編集', 'CHAP

In [30]:
titles

['急成長を導くマネージャーの型～地位・権力が通用しない時代の“イーブン”なマネジメント',
 'ゼロからはじめるiPhone 12 スマートガイド iOS 15対応版',
 'ゼロからはじめるiPhone SE 第2世代 スマートガイド iOS 15対応版',
 'ゼロからはじめるiPhone 13 mini スマートガイド ドコモ完全対応版',
 'ゼロからはじめるiPhone 13 Pro/Pro Max スマートガイド ドコモ完全対応版',
 '今すぐ使えるかんたんExPowerPoint ビジネス作図 プロ技BESTセレクション',
 'ゼロからはじめるiPhone 13/mini/Pro/Pro Max スマートガイド ソフトバンク完全対応版',
 'ゼロからはじめるiPhone 13 スマートガイド ドコモ完全対応版',
 'ゼロからはじめるiPhone 13/mini/Pro/Pro Max スマートガイド au完全対応版',
 'Windows 10→Windows 11 乗り換え&徹底活用 100％入門ガイド',
 '最短突破Microsoft Azure Fundamentals［AZ-900］合格教本',
 '改訂新版 電子工作の素',
 'ISOマネジメントシステムが一番わかる',
 '大きな字でわかりやすいTwitter ツイッター入門',
 'WEB+DB PRESS Vol.125',
 '一気にビギナー卒業！ 動画でわかるAfter Effects教室',
 'AutoCAD パーフェクトガイド［改訂2版］',
 'PowerPoint［最強］時短仕事術 もう迷わない！ ひと目で伝わる資料作成',
 '「会社は無理ゲー」な人がノビノビ稼ぐ方法',
 '働き方のデジタルシフト——リモートワークからはじめる、しなやかな組織づくりの処方箋',
 'アートで魅せる数学の世界',
 '今すぐ使えるかんたんAutoCAD［改訂2版］',
 '初心者でもできる！ せどり副業で月収10万円',
 'Software Design 2021年11月号',
 'Amazon出店の王道～中小企業の新販路 その施策とアイデア',
 '秒速決算～スピーディに人を動かす管理会計で最高の利益体質をつくる！～',
 'EDIUS X Pro パーフェクトガイド［改

In [31]:
len(titles)

30

In [22]:
def connect_gspread(jsonf,key):
    #spreadsheetsとdriveの2つのAPIを指定する
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    #認証情報を設定する
    credentials = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
    gc = gspread.authorize(credentials)
    #スプレッドシートキーを用いて、sheet1にアクセスする
    SPREADSHEET_KEY = key
    worksheet = gc.open_by_key(SPREADSHEET_KEY).sheet1
    return worksheet

In [34]:
# jsonfile名を指定
jsonf = "onyx-link-331410-ad335d7b2a4a.json"
# 共有設定したスプレットシートキーを指定
spread_sheet_key = "1frwEfXRcIJLcLesebojfkBqC2WgEpOBzDgFdG500SVo"
ws = connect_gspread(jsonf,spread_sheet_key)

In [35]:
cell_list = ws.range('A1:A30')
i = 0
for cell in cell_list:
    cell.value = titles[i]
    i += 1
ws.update_cells(cell_list)

{'spreadsheetId': '1frwEfXRcIJLcLesebojfkBqC2WgEpOBzDgFdG500SVo',
 'updatedRange': "'シート1'!A1:A30",
 'updatedRows': 30,
 'updatedColumns': 1,
 'updatedCells': 30}

重複に注意
データベースは３つある

In [1]:
a=['a']
b=['b']
c=[]
c.append(a)
c.append(b)
c

[['a'], ['b']]